#  Feature vectorization 개요
- 텍스트를 숫자형값의 정형테이터로 변환하는 것을 Feature vectorization(피처 벡터화) 라고 한다.

## BOW (Bag Of Words)
**많이 나온 단어가 중요한 단어**
- 문서내에 단어 빈도수에 기반하여 Vector화 하는 모델
- DTM/TDM (Document Term Matrix)  // D가 먼저나오니까 행이 문서 
    - 문서안에서 문서를 구성하는 단어들이 몇번 나왔는지를 표현하는 행렬
    - 행:단어, 열: 문서 - TDM
    - 행:문서, 열:단어 - DTM
    - Value: 개수
- TF-IDF (Term Frequency Inverse Document Frequency)
    - CountVectorize의 문제: 문장 구조상 많이 나오는 단어들의 경우 카운트 값이 많이 나오게 되고 중요한 단어로 인식된다. (ex: 관사, 접속사, 주제어 등) 이 문제를 보완한 모델이 TF-IDF 모델이다.
    - 개별 문서에 많이 나오는 단어가 높은 값을 가지도록 하되 동시에 ***여러 문서에 자주 나오는 단어에는 페널티를 주는 방식***(문서를 구분하는데에는 중요한 역할을 하지 못하니까.)

## DTM/TDM (Document Term Matrix)

- 단어들이 각 문서에 몇번 나왔는지 행렬로 구성
- 많이 나온 단어가 중요한 단어라는 것을 기반으로 한다.
- 문서 단어 행렬(Document Term Matrix)
    - 문서별로 각 단어가 나타난 **횟수를 정리한 표**
    - 컬럼(Feature): 전체 문서에 나오는 모든 단어
    - 행 : 문서
    - 값(value) : 각 단어가 문서에 나온 횟수
- 단어 문서 행렬(Term Document Matrix)
    - DTM을 전치 시킨 것
    - 컬럼(Feature): 문서
    - 행: 전체 문서에 나오는 모든 단어
    - 값(value) : 각 단어가 문서에 나온 횟수
- scikit-learn의 CountVectorize 이용

## CountVectorizer 
#### 주요 생성자 매개변수
- stop_word :stopword 지정 
    - str: "english" - 영문 불용어는 제공됨
    - list: stopword 리스트
- max_df: 특정 횟수 이상나오는 것은 무시하도록 설정(무시할 횟수/비율 지정)
    - int(횟수), float(비율)
- min_df: 특정 횟수 이하로 나오는 것은 무시하도록 설정(무시할 횟수/비율 지정)
- max_features: 최대 token 수
    - 빈도수가 높은 순서대로 정렬 후 지정한 max_features 개수만큼만 사용한다.
- ngram_range: n_gram 범위 지정  // n 값이 커지면 성능이 좋아지지만 시간이 너무 많이 걸린다.// 디폴트는 유니그램 1개 씩만 묶기
    - n_gram:
    - 튜플 (범위 최소값, 범위 최대값)
    - (1, 2) : 토큰화된 단어를 1개씩 그리고 순서대로 2개씩 묶어서 Feature를 추출

#### 메소드
- fit(X)
    - 학습
    - 매개변수: raw document - 문장을 원소로 가지는 1차원 배열형태(list, ndarray) --> 문
    - **Train(훈련) 데이터셋 으로 학습한다. Test 데이터셋은 Train 셋으로 학습한 CountVectorizer를 이용해 변환만 한다.**
- transform(X)
    - DTM 변환
- fit_transform(X)
    - 학습/변환 한번에 처리

### n-gram
DTM는 문맥상에서 단어의 의미는 무시된다. 이것을 보완하는 것이 n-gram 기법이다.   
n개의 단어를 묶어서 feature로 구성한다.

In [187]:
train = ["He really likes football. He wants to be a football player.",   # 원소 : 문서이다 // train 문서 3개 이다.
         "Football is a popular sport in Europe.", 
         "Which country started football?"]
test = ["He really likes baseball. He wants to be a baseball player.", 
        "Baseball is a popular sport in Korea."]

In [188]:
data = train + test  # g합쳐서 train 시키기 위해서..
data

['He really likes football. He wants to be a football player.',
 'Football is a popular sport in Europe.',
 'Which country started football?',
 'He really likes baseball. He wants to be a baseball player.',
 'Baseball is a popular sport in Korea.']

### train 셋을 이용해 학습 및 변환

In [189]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
# cv = CountVectorizer(ngram_range =(1,3))# 점점더 문서와 유사해지면서 성능은 높아지지만 속도가 매우 느려진다. 컬럼이 많아져서
#cv = CountVectorizer(ngram_range =(3,3))
cv.fit(data) # 학습은 train, test를 합쳐서 (나누기 전에 해야한다.)
train_cv= cv.transform(train)


In [190]:
train_cv.toarray()

array([[0, 1, 0, 0, 2, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]],
      dtype=int64)

In [191]:
print(cv.get_feature_names())  # 변환할때 소문자로 변환시켜줌

['baseball', 'be', 'country', 'europe', 'football', 'he', 'in', 'is', 'korea', 'likes', 'player', 'popular', 'really', 'sport', 'started', 'to', 'wants', 'which']


In [197]:
pd.DataFrame(train_cv, columns = cv.get_feature_names())

ValueError: Shape of passed values is (3, 1), indices imply (3, 18)

# 원래는 테스트 데이터 셋을 나누기 전에 countvectorize 학습을 시켰어야 한다.
 ### trainset에 있는 단어들만 count 할 수 있다... test 엔 야구가 잇는데 카운트가 안됨

### test 셋 변환

In [193]:
test_cv =cv.transform(test).toarray()  # trainset에 있는 단어들만 count 할 수 있다... test 엔 야구가 잇는데 카운트가 안됨
test_cv.shape 

(2, 18)

In [194]:
test_cv

array([[2, 1, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]],
      dtype=int64)

In [195]:
import pandas as pd
pd.DataFrame(test_cv, columns = cv.get_feature_names()) # trainset에 있는 단어들만 count 할 수 있다... test 엔 baseball이 잇는데 카운트가 안됨

,baseball,be,country,europe,football,he,in,is,korea,likes,player,popular,really,sport,started,to,wants,which
0,2,1,0,0,0,2,0,0,0,1,1,0,1,0,0,1,1,0
1,1,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0


## TF-IDF (Term Frequency - Inverse Document Frequency)
- 개별 문서에 많이 나오는 단어가 높은 값을 가지도록 하되 동시에 ***여러 문서에 자주 나오는 단어에는 페널티를 주는 방식**
- 어떤 문서에 특정 단어가 많이 나오면 그 단어는 해당 문서를 설명하는 중요한 단어일 수 있다. 그러나 그 단어가 다른 문서에도 많이 나온다면 언어 특성이나 주제상 많이 사용되는 단어 일 수 있다.
    - 전체 문서에 고르게 많이 나오는 단어들은 각각의 문서가 다른 문서와 다른 특징을 찾는데 도움이 안된다. 그래서 페널티를 주어 작은 값이 되도록 한다.
- 각 문서의 길이가 길고 문서개수가 많은 경우 Count 방식 보다 TF-IDF 방식이 더 좋은 예측 성능을 내는 경우가 많다.

### 공식
- TF (Term Frequency) : 해당 단어가 해당 문서에 몇번 나오는지를 나타내는 지표
- DF (Document Frequency) : 해당 단어가 몇개의 문서에 나오는지를 나타내는 지표// 단어출연문서개수/전체문서 개수
- IDF (Inverse Document Frequency) : DF에 역수로 $\cfrac{전체 문서수}{해당단어가 나오는 문서수}$
- TF-IDF : $TF * \left(\log \cfrac{전체 문서수}{해당단어가 나오는 문서수} \right)$

### TfidfVectorizer


#### 주요 생성자 매개변수
- stop_word :stopword 지정 
    - str: "english" - 영문 불용어는 제공됨
    - list: stopword 리스트
- max_df: 특정 횟수 이상나오는 것은 무시하도록 설정(무시할 횟수/비율 지정)
    - int(횟수), float(비율)
- min_df: 특정 횟수 이하로 나오는 것은 무시하도록 설정(무시할 횟수/비율 지정)
- max_features: 최대 token 수
    - 빈도수가 높은 순서대로 정렬 후 지정한 max_features 개수만큼만 사용한다.
- ngram_range: n_gram 범위 지정
    - n_gram:
    - 튜플 (범위 최소값, 범위 최대값)
    - (1, 2) : 토큰화된 단어를 1개씩 그리고 순서대로 2개씩 묶어서 Feature를 추출

#### 메소드
- fit(X)
    - 학습
    - 매개변수: 문장을 가진 1차원 배열형태(list, ndarray) 
    - **Train(훈련)+test(테스트) 데이터셋 으로 학습한다.***
- transform(X)
    - DTM 변환
- fit_transform(X)
    - 학습/변환 한번에 처리

In [102]:
train = ["He really likes football. He wants to be a football player.", 
         "Football is a popular sport in Europe.", 
         "Which country started football?"]
test = ["He really likes baseball. He wants to be a baseball player.", 
        "Baseball is a popular sport in Korea."]

In [103]:
data = train+test 

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [105]:
tfidf = TfidfVectorizer(stop_words = 'english')
#tfidf = TfidfVectorizer(stop_words = 'english', ngram_range=(1,3))  # 원하는 것으로 설정가능 

In [106]:
tfidf.fit(data)

TfidfVectorizer(stop_words='english')

In [107]:
print(len(tfidf.get_feature_names()))
tfidf.get_feature_names()  # 셀 단어 , 단어 뭉치들~!// 어간 추출을 해줘야 하는데 함수로 만들어서 해야함 ~!

12


['baseball',
 'country',
 'europe',
 'football',
 'korea',
 'likes',
 'player',
 'popular',
 'really',
 'sport',
 'started',
 'wants']

In [108]:
train_tfidf = tfidf.transform(train)
test_tfidf = tfidf.transform(test)

In [109]:
pd.DataFrame(train_tfidf.toarray(), columns = tfidf.get_feature_names())

,baseball,country,europe,football,korea,likes,player,popular,really,sport,started,wants
0,0.0,0.00000,0.000000,0.638711,0.0,0.384724,0.384724,0.000000,0.384724,0.000000,0.00000,0.384724
1,0.0,0.00000,0.602985,0.403826,0.0,0.000000,0.000000,0.486484,0.000000,0.486484,0.00000,0.000000
2,0.0,0.63907,0.000000,0.427993,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.63907,0.000000


# IMDB(Internet Movie Database)  영화리뷰 데이터 셋
- https://www.imdb.com/
- 다운로드: http://ai.stanford.edu/~amaas/data/sentiment/
    - train의 unsup 은 제거 (비지도학습용)

- load_files() 
    - 분류범주를 폴더로 분리한 텍스트 파일을 load한다.
    - Bunch 타입으로 반환
    - 라벨이 되는 디렉토리(폴더)구조를 잘만들어줘야한다.!

In [110]:
from sklearn.datasets import load_files
import numpy as np
import pandas as pd

In [111]:
review_train = load_files('aclImdb/train') # 경로 넣어줌 
review_test = load_files('aclImdb/test')

In [112]:
type(review_train)

sklearn.utils.Bunch

In [113]:
review_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [114]:
review_train['target_names']

['neg', 'pos']

In [115]:
review_train['target']

array([1, 0, 1, ..., 0, 0, 0])

In [116]:
print(type(review_train['data']))
review_train['data'][:2]

<class 'list'>


[b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.",
 b'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many cli

In [120]:
len(review_train['data']),len(review_test['data'])

(25000, 25000)

In [122]:
np.unique(review_train['target'], return_counts =True)

(array([0, 1]), array([12500, 12500], dtype=int64))

- neg: 0, pos: 1 로 분리해 준다. (폴더의 알파벳 순서대로 )
- Bunch 타입으로 반환

## 전처리
- `<br/>` 제거
- binary string을 string으로 변환

In [130]:
import nltk
from nltk.stem import WordNetLemmatizer  # 원형복원
from nltk.tag import pos_tag # 원형복원하기위해 품사 붙이고 나중에 원형복원의 품사형식으로 바꿀것 
from nltk.corpus import stopwords


def get_wordnet_tagset(tag):
    
    if tag.startswith('J'):
        return 'a'
    elif tag.startswith('N'):
        return 'n'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('R'):
        return 'r'
    else:
        return None

def text_preprocessing(documents):
    """
    문서 리스트를 받아서 각각의 문서를 전처리 
    [매개변수]
         documents : list 문서 리스트 // 각각의 원소가  문서하나씩 
    [반환값]
         list: 전처리된 문서들을 가진 리스트.
    """
    lemm = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    
    return_list =[]
    for document in documents:
        document = document.decode('UTF-8')  # 앞에 b 가 붙어있다 바이너리 데이터 이기때문에 docode utf-8을 해줘야 텍스트로 바뀜
        # 소문자로 변환
        document = document.lower()
        #<br /> 태그 제거 # 공백으로 바꿈
        document = document.replace('<br />'," ") 
        
        tokens = nltk.regexp_tokenize(document, r'[A-Za-z]+')
        
        # 불용어 제거
        token = [token for token in tokens if token not in stop_words]
        
        # 원형복원
        tokens = pos_tag(tokens)
        tokens = [ lemm.lemmatize(word, pos=get_wordnet_tagset(tag)) for word, tag in tokens if get_wordnet_tagset(tag) != None]
        
        return_list.append(" ".join(tokens))
        #
    
    
    return return_list

In [131]:
X_train = text_preprocessing(review_train['data'])
y_train = review_train['target']

In [198]:
len(X_train)

25000

In [182]:
type(y_train)

numpy.ndarray

In [132]:
X_test = text_preprocessing(review_test['data'])
y_test = review_test['target']

### 전처리한 데이터셋 파일로 저장

In [133]:
import os # 저장할 디렉 토리 만들때 한번에 처리 하기위해
import pickle
path ='imdb_dataset'

if not os.path.isdir(path): # path 경로가 없다면~  os.mkdir(path='imdb_dataset') 'imdb_dataset'디렉토리를 만들어라~
    os.mkdir(path)

In [134]:
with open(os.path.join(path, 'x_train.pkl'), 'wb') as f:
    pickle.dump(X_train, f)

In [136]:
with open(os.path.join(path, 'y_train.pkl'), 'wb') as f:
    pickle.dump(y_train,f)

In [137]:
with open(os.path.join(path, 'x_test.pkl'), 'wb') as f:
    pickle.dump(X_test,f)

In [138]:
with open(os.path.join(path, 'y_test.pkl'), 'wb') as f:
    pickle.dump(y_test,f)

### 읽기

In [139]:
with open(os.path.join(path, 'x_train.pkl'), 'rb') as f:  # 읽으니까 rb
    X_train_tmp = pickle.load(f)

In [142]:
X_train_tmp[0]

'day lead think even re think boy young men do do commit mutual suicide slaughter classmate capture be bizarre mode be human have decide withdraw common civility order define own mutual world couple destruction be not perfect movie give money time filmmaker actor have be remarkable product term explain motif action young suicide murderer be well elephant term be film get rationalistic skin be far far good film almost anything be likely see flawed honest terrible honesty'

## Feature Vectorization 

### DTM
- countvectorize

In [161]:
X_train # w전처리된 데이터를 CountVectorizer 객체에 학습시킨다.

['day lead think even re think boy young men do do commit mutual suicide slaughter classmate capture be bizarre mode be human have decide withdraw common civility order define own mutual world couple destruction be not perfect movie give money time filmmaker actor have be remarkable product term explain motif action young suicide murderer be well elephant term be film get rationalistic skin be far far good film almost anything be likely see flawed honest terrible honesty',
 'word t describe bad movie be i t explain write only have too see get grip horrible movie really be not i recommend do be so many clich s mistake other negative thing imagine here just make cry start technical first be lot mistake regard airplane i win t list here just mention coloring plane didn t even manage show airliner color fictional airline instead use paint original boeing livery very bad plot be stupid have be do many time only much much good be so many ridiculous moment here i lose count really early also 

In [146]:
# 전처리된 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df = 10)  # 불용어 전처리에서 없앰
cv.fit(X_train+X_test)
len(cv.get_feature_names())

20683

In [148]:
X_train_dtm = cv.transform(X_train)
X_test_dtm = cv.transform(X_test)

In [149]:
X_train_dtm.shape, X_test_dtm.shape

((25000, 20683), (25000, 20683))

In [164]:
X_train_dtm.toarray()# 각행(문서)마다 단어 있는 횟수 만큼 숫자가 있을것이다.

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### 머신러닝 알고리즘을 이용해 긍부정 분류

In [153]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 1000)

# 학습

lr.fit(X_train_dtm,y_train)  #y_train은 이미 0,1 로 되어있고 , X_train_dtm은 전처리와 countvectorize를 통해 변환된것

LogisticRegression(max_iter=1000)

In [155]:
pred_train = lr.predict(X_train_dtm)
pred_test = lr.predict(X_test_dtm)


In [156]:
from sklearn.metrics import accuracy_score
print('Train정확도: ', accuracy_score(y_train, pred_train))
print('test정확도: ',  accuracy_score(y_test, pred_test))

Train정확도:  0.994
test정확도:  0.85464


In [158]:
tmp = cv.transform(['i hate this movie'])
lr.predict(tmp.reshape(1,-1))  # 틀림 

array([1])

### TF-IDF

In [168]:
tfidf = TfidfVectorizer(max_features =20000)

tfidf.fit(X_train+X_test)  # 위에 프리프로세싱 함수로 한것을 쓴것이다.

TfidfVectorizer(max_features=20000)

In [169]:
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [171]:
X_train_tfidf.shape,X_test_tfidf.shape

((25000, 20000), (25000, 20000))

### 머신러닝 알고리즘을 이용해 긍부정 분류

In [172]:
lr2 = LogisticRegression(max_iter = 1000)
lr2.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [174]:
pred_train2 = lr2.predict(X_train_tfidf)
pred_test2 = lr2.predict(X_test_tfidf)

In [175]:
accuracy_score(y_train,pred_train2), accuracy_score(y_test,pred_test2)

(0.92536, 0.87888)

In [176]:
# tree 모델로 하기// 성능 떨어짐 .. 희소행렬 사용해서.... 할때 tree모델...

In [177]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators= 500, max_depth = 3)

rf.fit(X_train_tfidf,y_train)

RandomForestClassifier(max_depth=3, n_estimators=500)

In [178]:
pred_train3 = rf.predict(X_train_tfidf)
pred_test3 = rf.predict(X_test_tfidf)

In [179]:
accuracy_score(y_train, pred_train3), accuracy_score(y_test, pred_test3)

(0.8374, 0.82368)